In [10]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import concat_ws, col, when,lit,concat,monotonically_increasing_id

In [4]:
spark = SparkSession.builder \
    .appName('AppyLogicalinDataframe') \
    .getOrCreate()

24/03/31 19:34:35 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.sql.repl.eagerEval.maxNumRows", 1000)
spark.conf.set("spark.sql.repl.eagerEval.truncate", 1000)

In [3]:
schema = StructType([
    StructField("COLUMN1", StringType(), True),
    StructField("COLUMN2", StringType(), True),
    StructField("COLUMN3", StringType(), True),
    StructField("COLUMN4", StringType(), True),
    StructField("DATOS_BASICOS", StringType(), True)
])

# Create data
data = [("CAMILA OR MARTIN OR PEDRO OR PEPE OR GLORIA CHOMALI,OR TERESA CHOMALI","OR LUISA OR CARMEN OR","ALEJANDRA","AV CARMEN","INFORMACION PERSONAL"),
        ("CAMILA OR MARTIN","AV 1","AV 2","AV CARMEN","INFORMACION PERSONAL")]

# Create DataFrame
df = spark.createDataFrame(data, schema=schema)

# Show DataFrame
df

DataFrame[COLUMN1: string, COLUMN2: string, COLUMN3: string, COLUMN4: string, DATOS_BASICOS: string]

In [6]:
df

COLUMN1,COLUMN2,COLUMN3,COLUMN4,DATOS_BASICOS
"CAMILA OR MARTIN OR PEDRO OR PEPE OR GLORIA CHOMALI,OR TERESA CHOMALI",OR LUISA OR CARMEN OR,ALEJANDRA,AV CARMEN,INFORMACION PERSONAL
CAMILA OR MARTIN,AV 1,AV 2,AV CARMEN,INFORMACION PERSONAL


In [7]:
column_names = ["COLUMN1","COLUMN2","COLUMN3","COLUMN4"]
df = df.withColumn("ID",monotonically_increasing_id())

In [353]:
for i in range(2, len(column_names)):
    prev_col = column_names[i-1]
    current_col = column_names[i]
    df = df.withColumn(current_col, F.when(F.col(prev_col).rlike(".*OR$"), F.concat(F.lit('OR '), F.col(current_col))).otherwise(F.col(current_col)))

In [11]:
df = df.withColumn("LISTA_CLIENTES", F.lit(''))

for i in column_names:
    df = df.withColumn("LISTA_CLIENTES", F.when(F.col(i).rlike(".*OR.*"), F.concat(F.col("LISTA_CLIENTES"), F.col(i), F.lit(' '))).otherwise(F.col("LISTA_CLIENTES")))

In [13]:
df = df.withColumn("LISTA_CLIENTES", F.regexp_replace(F.col("LISTA_CLIENTES"), ",OR", " OR"))

In [14]:
df

COLUMN1,COLUMN2,COLUMN3,COLUMN4,DATOS_BASICOS,ID,LISTA_CLIENTES
"CAMILA OR MARTIN OR PEDRO OR PEPE OR GLORIA CHOMALI,OR TERESA CHOMALI",OR LUISA OR CARMEN OR,ALEJANDRA,AV CARMEN,INFORMACION PERSONAL,25769803776,CAMILA OR MARTIN OR PEDRO OR PEPE OR GLORIA CHOMALI OR TERESA CHOMALI OR LUISA OR CARMEN OR
CAMILA OR MARTIN,AV 1,AV 2,AV CARMEN,INFORMACION PERSONAL,60129542144,CAMILA OR MARTIN


In [15]:
df = df.withColumn("LISTA_CLIENTES", F.split(F.col("LISTA_CLIENTES"), " OR "))

In [303]:
#df = df.withColumn("LISTA_CLIENTES", F.explode(F.col("LISTA_CLIENTES")))

In [16]:
df = df.select("*", F.posexplode(F.col("LISTA_CLIENTES")).alias("CONSECUTIVO", "CLIENTE"))

In [17]:
df

COLUMN1,COLUMN2,COLUMN3,COLUMN4,DATOS_BASICOS,ID,LISTA_CLIENTES,CONSECUTIVO,CLIENTE
"CAMILA OR MARTIN OR PEDRO OR PEPE OR GLORIA CHOMALI,OR TERESA CHOMALI",OR LUISA OR CARMEN OR,ALEJANDRA,AV CARMEN,INFORMACION PERSONAL,25769803776,"[CAMILA, MARTIN, PEDRO, PEPE, GLORIA CHOMALI, TERESA CHOMALI, LUISA, CARMEN, ]",0,CAMILA
"CAMILA OR MARTIN OR PEDRO OR PEPE OR GLORIA CHOMALI,OR TERESA CHOMALI",OR LUISA OR CARMEN OR,ALEJANDRA,AV CARMEN,INFORMACION PERSONAL,25769803776,"[CAMILA, MARTIN, PEDRO, PEPE, GLORIA CHOMALI, TERESA CHOMALI, LUISA, CARMEN, ]",1,MARTIN
"CAMILA OR MARTIN OR PEDRO OR PEPE OR GLORIA CHOMALI,OR TERESA CHOMALI",OR LUISA OR CARMEN OR,ALEJANDRA,AV CARMEN,INFORMACION PERSONAL,25769803776,"[CAMILA, MARTIN, PEDRO, PEPE, GLORIA CHOMALI, TERESA CHOMALI, LUISA, CARMEN, ]",2,PEDRO
"CAMILA OR MARTIN OR PEDRO OR PEPE OR GLORIA CHOMALI,OR TERESA CHOMALI",OR LUISA OR CARMEN OR,ALEJANDRA,AV CARMEN,INFORMACION PERSONAL,25769803776,"[CAMILA, MARTIN, PEDRO, PEPE, GLORIA CHOMALI, TERESA CHOMALI, LUISA, CARMEN, ]",3,PEPE
"CAMILA OR MARTIN OR PEDRO OR PEPE OR GLORIA CHOMALI,OR TERESA CHOMALI",OR LUISA OR CARMEN OR,ALEJANDRA,AV CARMEN,INFORMACION PERSONAL,25769803776,"[CAMILA, MARTIN, PEDRO, PEPE, GLORIA CHOMALI, TERESA CHOMALI, LUISA, CARMEN, ]",4,GLORIA CHOMALI
"CAMILA OR MARTIN OR PEDRO OR PEPE OR GLORIA CHOMALI,OR TERESA CHOMALI",OR LUISA OR CARMEN OR,ALEJANDRA,AV CARMEN,INFORMACION PERSONAL,25769803776,"[CAMILA, MARTIN, PEDRO, PEPE, GLORIA CHOMALI, TERESA CHOMALI, LUISA, CARMEN, ]",5,TERESA CHOMALI
"CAMILA OR MARTIN OR PEDRO OR PEPE OR GLORIA CHOMALI,OR TERESA CHOMALI",OR LUISA OR CARMEN OR,ALEJANDRA,AV CARMEN,INFORMACION PERSONAL,25769803776,"[CAMILA, MARTIN, PEDRO, PEPE, GLORIA CHOMALI, TERESA CHOMALI, LUISA, CARMEN, ]",6,LUISA
"CAMILA OR MARTIN OR PEDRO OR PEPE OR GLORIA CHOMALI,OR TERESA CHOMALI",OR LUISA OR CARMEN OR,ALEJANDRA,AV CARMEN,INFORMACION PERSONAL,25769803776,"[CAMILA, MARTIN, PEDRO, PEPE, GLORIA CHOMALI, TERESA CHOMALI, LUISA, CARMEN, ]",7,CARMEN
"CAMILA OR MARTIN OR PEDRO OR PEPE OR GLORIA CHOMALI,OR TERESA CHOMALI",OR LUISA OR CARMEN OR,ALEJANDRA,AV CARMEN,INFORMACION PERSONAL,25769803776,"[CAMILA, MARTIN, PEDRO, PEPE, GLORIA CHOMALI, TERESA CHOMALI, LUISA, CARMEN, ]",8,
CAMILA OR MARTIN,AV 1,AV 2,AV CARMEN,INFORMACION PERSONAL,60129542144,"[CAMILA, MARTIN ]",0,CAMILA


In [ ]:
#df = df.withColumn("Name", when(col("Name").endswith("a"), concat(col("Name"), lit("b"))).otherwise(col("Name")))

In [375]:
from pyspark.sql import functions as F
from pyspark.sql.functions import monotonically_increasing_id

def process_dataframe(df, column_names):
    df = df.withColumn("ID", monotonically_increasing_id())

    for i in range(2, len(column_names)):
        prev_col = column_names[i-1]
        current_col = column_names[i]
        df = df.withColumn(current_col, F.when(F.col(prev_col).rlike(".*OR$"), F.concat(F.lit('OR '), F.col(current_col))).otherwise(F.col(current_col)))

    df = df.withColumn("LISTA_CLIENTES", F.lit(''))

    for i in column_names:
        df = df.withColumn("LISTA_CLIENTES", F.when(F.col(i).rlike(".*OR.*"), F.concat(F.col("LISTA_CLIENTES"), F.col(i), F.lit(' '))).otherwise(F.col("LISTA_CLIENTES")))

    df = df.withColumn("LISTA_CLIENTES", F.regexp_replace(F.col("LISTA_CLIENTES"), "OR OR", "OR"))

    df = df.withColumn("LISTA_CLIENTES", F.split(F.col("LISTA_CLIENTES"), " OR "))

    df = df.select("*", F.posexplode(F.col("LISTA_CLIENTES")).alias("CONSECUTIVO", "CLIENTE"))

    return df

In [376]:
column_names = ["COLUMN1","COLUMN2","COLUMN3","COLUMN4"]
df = process_dataframe(df, column_names)

In [377]:
df

COLUMN1,COLUMN2,COLUMN3,COLUMN4,DATOS_BASICOS,ID,LISTA_CLIENTES,CONSECUTIVO,CLIENTE
CAMILA OR MARTIN OR PEDRO OR PEPE,OR LUISA OR CARMEN OR,OR ALEJANDRA,AV CARMEN,INFORMACION PERSONAL,25769803776,"[CAMILA, MARTIN, PEDRO, PEPE, LUISA, CARMEN, ALEJANDRA ]",0,CAMILA
CAMILA OR MARTIN OR PEDRO OR PEPE,OR LUISA OR CARMEN OR,OR ALEJANDRA,AV CARMEN,INFORMACION PERSONAL,25769803776,"[CAMILA, MARTIN, PEDRO, PEPE, LUISA, CARMEN, ALEJANDRA ]",1,MARTIN
CAMILA OR MARTIN OR PEDRO OR PEPE,OR LUISA OR CARMEN OR,OR ALEJANDRA,AV CARMEN,INFORMACION PERSONAL,25769803776,"[CAMILA, MARTIN, PEDRO, PEPE, LUISA, CARMEN, ALEJANDRA ]",2,PEDRO
CAMILA OR MARTIN OR PEDRO OR PEPE,OR LUISA OR CARMEN OR,OR ALEJANDRA,AV CARMEN,INFORMACION PERSONAL,25769803776,"[CAMILA, MARTIN, PEDRO, PEPE, LUISA, CARMEN, ALEJANDRA ]",3,PEPE
CAMILA OR MARTIN OR PEDRO OR PEPE,OR LUISA OR CARMEN OR,OR ALEJANDRA,AV CARMEN,INFORMACION PERSONAL,25769803776,"[CAMILA, MARTIN, PEDRO, PEPE, LUISA, CARMEN, ALEJANDRA ]",4,LUISA
CAMILA OR MARTIN OR PEDRO OR PEPE,OR LUISA OR CARMEN OR,OR ALEJANDRA,AV CARMEN,INFORMACION PERSONAL,25769803776,"[CAMILA, MARTIN, PEDRO, PEPE, LUISA, CARMEN, ALEJANDRA ]",5,CARMEN
CAMILA OR MARTIN OR PEDRO OR PEPE,OR LUISA OR CARMEN OR,OR ALEJANDRA,AV CARMEN,INFORMACION PERSONAL,25769803776,"[CAMILA, MARTIN, PEDRO, PEPE, LUISA, CARMEN, ALEJANDRA ]",6,ALEJANDRA
CAMILA OR MARTIN,AV 1,AV 2,AV CARMEN,INFORMACION PERSONAL,60129542144,"[CAMILA, MARTIN ]",0,CAMILA
CAMILA OR MARTIN,AV 1,AV 2,AV CARMEN,INFORMACION PERSONAL,60129542144,"[CAMILA, MARTIN ]",1,MARTIN


In [18]:
to_timestamp

NameError: name 'to_timestamp' is not defined